In [1]:
import sqlite3
import pandas as pd
import uuid

#import sys
#sys.path.append('/home/debiani/local_dev/reis-app/src/lib')

import os
from dotenv import load_dotenv

load_dotenv()
DB_PATH = os.getenv("DATABASE_URL", "sqlite.db")

In [2]:
df = pd.read_excel('./ZBE-cat.xlsx')

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.replace(r"\s+", " ", regex=True)

# Eliminar espacios al principio y al final de las celdas de texto
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# define columns
cols = ['Grup', 'Question']

df = df[cols]
df

,Grup,Question
0,amoonguss,S’hauria d’ampliar o disminuir la superfície d...
1,amoonguss,Quin efecte han tingut les ZBE?
2,amoonguss,S’ha reduït la contaminació des de la implemen...
3,amoonguss,Quins efectes té l’acumulació de NO2 en aquest...
4,amoonguss,Quines són les zones de baixes emissions defin...
...,...,...
89,weaviles,Quin tipus de combustible és més contaminant?
90,weaviles,Estan correctament establertes les normes de l...
91,weaviles,A partir de quina antiguitat els vehícles cont...
92,weaviles,Quin és la relació entre les superfícies de le...


In [3]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Crear tabla class
cursor.execute("""
    CREATE TABLE IF NOT EXISTS class (
        id TEXT PRIMARY KEY NOT NULL,
        title TEXT NOT NULL,
        desc TEXT,
        entercode TEXT NOT NULL,
        start_date TEXT NOT NULL DEFAULT current_timestamp,
        end_date TEXT,
        deleted INTEGER NOT NULL DEFAULT 0,
        recorrido_id TEXT NOT NULL
    )
""")

# Crear tabla group
cursor.execute("""
    CREATE TABLE IF NOT EXISTS "group" (
        id TEXT PRIMARY KEY NOT NULL,
        lider_id TEXT NOT NULL,
        deleted INTEGER NOT NULL DEFAULT 0,
        recorrido_id TEXT NOT NULL
    )
""")

# Crear tabla user
cursor.execute("""
    CREATE TABLE IF NOT EXISTS user (
        id TEXT PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        email TEXT,
        emailVerified INTEGER,
        image TEXT,
        role TEXT NOT NULL DEFAULT 'user',
        last_seen TEXT NOT NULL DEFAULT current_timestamp,
        deleted INTEGER NOT NULL DEFAULT 0,
        password TEXT
    )
""")

# Crear tabla dx_question
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dx_question (
        id TEXT,
        question TEXT,
        class_id TEXT,
        group_id TEXT,
        dx_answer TEXT,
        feedback TEXT,
        parent_id TEXT
    )
""")

rei_id = 'a40e039f-c630-4d30-a4db-e812e3106568'
period = 'cdbb4989-6aa7-4663-8dcf-5cee59a6f54b'
# Insertar datos en la tabla class
cursor.execute("""
    INSERT OR IGNORE INTO class (id, title, desc, entercode, start_date, end_date, deleted, recorrido_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", (period, 'ZBE', 'ZBE', 'ZBE', '2023-10-01', None, 0, rei_id))

# Insertar datos en la tabla dx_question
for idx, row in df.iterrows():
    qn_id = str(uuid.uuid4())
    cursor.execute("""
            INSERT INTO dx_question (id, question, class_id, group_id, dx_answer, feedback, parent_id)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
        (qn_id, row['Question'], period, row['Grup'], '', None, None))

conn.commit()